In [ ]:
import torch
from torch import nn
import torchaudio
import torchvision

from sklearn.model_selection import train_test_split

from tqdm import tqdm
from sklearn import metrics

#from trainer import TorchSupervisedTrainer

import os

import pandas as pd
import json
import glob

import random
import os
import numpy as np

import argparse

from torchvision.transforms import v2

from datasets import MultimodalPhysVerbDataset, AggrBatchSampler, AppendZeroValues, MultimodalDataset
from trainer import MultimodalTrainer
from models import PhysVerbClassifierConcatFeatures, AveragedFeaturesTransformerFusion, PhysVerbClassifierAddFeatures, PhysVerbClassifier, PhysVerbModel, TransformerSequenceProcessor, EqualSizedTransformerModalitiesFusion, MultimodalModel, CNN1D, Swin3d_T_extractor, OutputClassifier, MultiModalCrossEntropyLoss, Wav2vec2Extractor, Wav2vecExtractor, AudioCnn1DExtractorWrapper


In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('--path_to_dataset',  required=True)
parser.add_argument('--path_to_intersections_csv')
parser.add_argument('--path_to_train_test_split_json')
parser.add_argument('--gpu_device_idx', type=int)
parser.add_argument('--class_num', type=int)
parser.add_argument('--resume_training', action='store_true')
parser.add_argument('--path_to_checkpoint')
parser.add_argument('--batch_size', required=True, type=int)
parser.add_argument('--nn_input_size', nargs='+', type=int)
parser.add_argument('--epoch_num', type=int)
parser.add_argument('--test_size', type=float)
parser.add_argument('--max_audio_len', type=int)
parser.add_argument('--max_embeddings_len', type=int)
parser.add_argument('--video_frames_num', type=int)
parser.add_argument('--video_window_size', type=int)

sample_args = [
    '--path_to_dataset',
    #r'/home/ubuntu/mikhail_u/DATASET_V0',
    #r'/home/aggr/mikhail_u/DATA/DATSET_V0',
    r'C:\Users\admin\python_programming\DATA\AVABOS\DATSET_V0',
    #r'I:\AVABOS\DATSET_V0',
    '--path_to_intersections_csv',
    #r'/home/ubuntu/mikhail_u/DATASET_V0/time_intervals_combinations_table.csv',
    #r'/home/aggr/mikhail_u/DATA/DATSET_V0/time_intervals_combinations_table.csv',
    r'C:\Users\admin\python_programming\DATA\AVABOS\DATSET_V0\time_intervals_combinations_table.csv',
    #r'i:\AVABOS\DATSET_V0\time_intervals_combinations_table.csv',
    '--path_to_train_test_split_json',
    r'train_test_split.json',
    '--gpu_device_idx', '0',
    '--class_num', '2',
    '--epoch_num', '100',
    '--batch_size', '64',
    '--max_audio_len', '80000',
    '--max_embeddings_len', '48',
    '--video_frames_num', '128',
    '--video_window_size', '8'
    ]

args = parser.parse_args(sample_args)

path_to_dataset_root = args.path_to_dataset
resume_training = args.resume_training
path_to_intersections_csv = args.path_to_intersections_csv
path_to_train_test_split_json = args.path_to_train_test_split_json

path_to_checkpoint = args.path_to_checkpoint
epoch_num = int(args.epoch_num)
class_num = args.class_num
batch_size = int(args.batch_size)
max_audio_len = args.max_audio_len
max_embeddings_len = args.max_embeddings_len
video_frames_num = args.video_frames_num
video_window_size = args.video_window_size
gpu_device_idx = args.gpu_device_idx

# имя модели соответствует имени экстрактора признаков
phys_gamma_val = 2
verb_gamma_val = 2
model_name = 'A+T(ce)+fusion1L'
modality2aggr = {'video':'phys', 'text':'verb', 'audio':'verb'}
#modality2aggr = {'video':'verb', 'text':'verb', 'audio':'phys'}
modalities_list = [
    'audio',
    'text',
    #'video'
    ]
aggr_types_list = set()
for m in modalities_list:
    aggr_types_list.add(modality2aggr[m])

aggr_types_list = list(aggr_types_list)

time_interval_combinations_df = pd.read_csv(path_to_intersections_csv)

with open(path_to_train_test_split_json) as fd:
    combinations_indices_dict = json.load(fd)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\admin\\python_programming\\DATA\\AVABOS\\DATSET_V0\\time_intervals_combinations_table.csv'